In [6]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [7]:
!pip install accelerate openai tiktoken transformers torch  retry scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

In [8]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.9 MB/s eta 0:00:00


In [9]:
import json
def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data
file_path = '/content/gdrive/My Drive/llama3_data/MAVEN/MAVEN.json'
json_data = read_json_file(file_path)
print(json_data[0])

{'title': '2006 Pangandaran earthquake and tsunami', 'id': '8307a6b61b84d4eea42c1dd5e6e2cdba', 'content': [{'sentence': 'The 2006 Pangandaran earthquake and tsunami occurred on July 17 at along a subduction zone off the coast of west and central Java, a large and densely populated island in the Indonesian archipelago.', 'tokens': ['The', '2006', 'Pangandaran', 'earthquake', 'and', 'tsunami', 'occurred', 'on', 'July', '17', 'at', 'along', 'a', 'subduction', 'zone', 'off', 'the', 'coast', 'of', 'west', 'and', 'central', 'Java', ',', 'a', 'large', 'and', 'densely', 'populated', 'island', 'in', 'the', 'Indonesian', 'archipelago', '.']}, {'sentence': 'The shock had a moment magnitude of 7.7 and a maximum perceived intensity of IV ("Light") in Jakarta, the capital and largest city of Indonesia.', 'tokens': ['The', 'shock', 'had', 'a', 'moment', 'magnitude', 'of', '7.7', 'and', 'a', 'maximum', 'perceived', 'intensity', 'of', 'IV', '(', '``', 'Light', "''", ')', 'in', 'Jakarta', ',', 'the', 'c

## Chọn mẫu ngẫu nhiên

In [10]:
import random

def get_random_samples(input_list, num_samples=20):
    """
    Get random samples from the input list.

    Args:
    input_list (list): The list to sample from.
    num_samples (int): The number of samples to return. Default is 20.

    Returns:
    list: A list of random samples.
    """
    if len(input_list) < num_samples:
        raise ValueError(f"Input list must have at least {num_samples} elements.")

    return random.sample(input_list, num_samples)


## Xử lý dữ liệu

In [11]:
sample_set = get_random_samples(json_data, num_samples=30)


def create_text_from_sample(data):
    # Ghép các từ trong mỗi câu
    sentences = [sentence["sentence"] for sentence in data['content']]
    return ' '.join(sentences)




In [17]:
def chunk_candidate_triggers(candidate_triggers, chunk_size):
    return [candidate_triggers[i:i + chunk_size] for i in range(0, len(candidate_triggers), chunk_size)]

def process_chunks(text, candidate_triggers, chunk_size, process_function):
    chunks = chunk_candidate_triggers(candidate_triggers, chunk_size)
    results = []

    for chunk in chunks:
        prompt = create_prompt(text, chunk)
        chunk_result = process_function(prompt)
        results.append(chunk_result)

    return merge_results(results)

def merge_results(results):
    merged = {"events": [], "negative_triggers": []}

    for result in results:
        for event in result.get("events", []):
            existing_event = next((e for e in merged["events"] if e["type"] == event["type"]), None)
            if existing_event:
                existing_event["mentions"].extend(event["mentions"])
            else:
                merged["events"].append(event)

        merged["negative_triggers"].extend(result.get("negative_triggers", []))

    # Remove duplicates
    for event in merged["events"]:
        event["mentions"] = list({(m["trigger_word"], m["sent_id"]): m for m in event["mentions"]}.values())

    merged["negative_triggers"] = list({(t["trigger_word"], t["sent_id"]): t for t in merged["negative_triggers"]}.values())

    return merged


In [12]:
def extract_events_and_neg_triggers(sample):
    sentences = [sentence["tokens"] for sentence in sample['content']]

    events = []
    triggers = []
    all_triggers = []
    for event in sample['events']:
        evt_type = event['type']
        mentions = []
        for mention in event["mention"]:
            mentions.append({'trigger_word': mention["trigger_word"], 'sent_id': mention["sent_id"]})
            all_triggers.append({'trigger_word': mention["trigger_word"], 'sent_id': mention["sent_id"]})
        events.append({'type': evt_type, 'mentions': mentions})

    for trigger in sample['negative_triggers']:
        triggers.append({'trigger_word': trigger['trigger_word'], 'sent_id': trigger['sent_id']})
        all_triggers.append({'trigger_word': trigger['trigger_word'], 'sent_id': trigger['sent_id']})

    return  all_triggers,{"events": events, "negative_triggers": triggers}




## Hàm tạo prompt

In [13]:
import json

def create_prompt(text, candidate_triggers):
    # Convert the candidate_triggers array to a JSON string
    triggers_json = json.dumps(candidate_triggers, indent=2)

    prompt = f"""
Analyze the following text and the provided list of candidate triggers to identify events and classify the triggers. Provide the information in this format:

1. Events: List events mentioned in the text. For each event, include:
   - Event type
   - Trigger words: List each as an object with:
     - trigger_word: The exact word from the candidate triggers list
     - sent_id: The sentence index as provided in the candidate triggers list

2. Negative triggers: Candidate triggers that don't actually indicate events in this context. Include each as an object with:
   - trigger_word: The exact word from the candidate triggers list
   - sent_id: The sentence index as provided in the candidate triggers list

Format your response in a JSON-like structure. Here's an example:

Text to analyze: The 2006 Pangandaran earthquake and tsunami occurred on July 17 along a subduction zone off the coast of west and central Java, a large and densely populated island in the Indonesian archipelago.

Candidate triggers: [
    {{"trigger_word": "earthquake", "sent_id": 0}},
    {{"trigger_word": "tsunami", "sent_id": 0}},
    {{"trigger_word": "occurred", "sent_id": 0}},
    {{"trigger_word": "subduction", "sent_id": 0}},
    {{"trigger_word": "populated", "sent_id": 0}},
    {{"trigger_word": "island", "sent_id": 0}}
]

Extracted results:
{{
  "events": [
    {{
      "type": "Catastrophe",
      "mentions": [
        {{"trigger_word": "earthquake", "sent_id": 0}},
        {{"trigger_word": "tsunami", "sent_id": 0}}
      ]
    }},
    {{
      "type": "Occurrence",
      "mentions": [
        {{"trigger_word": "occurred", "sent_id": 0}}
      ]
    }}
  ],
  "negative_triggers": [
    {{"trigger_word": "subduction", "sent_id": 0}},
    {{"trigger_word": "populated", "sent_id": 0}},
    {{"trigger_word": "island", "sent_id": 0}}
  ]
}}

Important guidelines:
1. Only consider the provided candidate triggers for event identification and negative trigger classification.
2. Classify each candidate trigger as either an event trigger or a negative trigger.
3. Use double quotes (") in the JSON structure.
4. Only extract information explicitly present in the text. Do not make predictions or inferences.
5. Use the exact trigger_word and sent_id as provided in the candidate triggers list.
6. Every candidate trigger must be classified either as part of an event or as a negative trigger.
7. Do not include any information other than the required JSON structure.

Now, please extract the events and classify the triggers for the following text and candidate triggers:
Text: {text}

Candidate triggers: {triggers_json}

Extracted Results:
"""
    return prompt

## Xử lý output

In [14]:
def parse_llm_output(output):
    # Tìm phần JSON trong đầu ra của LLM
    json_start = output.find('{')
    json_end = output.rfind('}') + 1
    json_str = output[json_start:json_end]

    parsed_json = json.loads(json_str)
    return parsed_json



## Hàm đánh giá kết quả

In [15]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import numpy as np
from scipy.spatial.distance import cosine
from difflib import SequenceMatcher


semantic_check_model_name = "bert-base-uncased"
semantic_check_tokenizer = AutoTokenizer.from_pretrained(semantic_check_model_name)
semantic_check_model = AutoModel.from_pretrained(semantic_check_model_name)



def calculate_text_similarity(text1, text2):
    return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()


def get_word_embedding(word):

    inputs = semantic_check_tokenizer(word, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = semantic_check_model(**inputs)

    return outputs.last_hidden_state[0][0].numpy()

def semantic_similarity(word1, word2):
    emb1 = get_word_embedding(word1)
    emb2 = get_word_embedding(word2)
    return 1 - cosine(emb1, emb2)

def is_event_similar(pred_event, true_event, threshold=0.8):
    """So sánh hai event dựa trên type và trigger_word."""
    type_similarity = semantic_similarity(pred_event['type'], true_event['type'])
    if type_similarity < threshold:
        return False

    pred_triggers = set(mention['trigger_word'] for mention in pred_event['mentions'])
    true_triggers = set(mention['trigger_word'] for mention in true_event['mentions'])

    return len(pred_triggers.intersection(true_triggers)) > 0

def calculate_event_f1(pred_output, true_output, threshold=0.8):
    """Tính F1 score cho events và negative triggers."""
    pred_events = pred_output['events']
    true_events = true_output['events']
    pred_neg_triggers = set(item['trigger_word'] for item in pred_output['negative_triggers'])
    true_neg_triggers = set(item['trigger_word'] for item in true_output['negative_triggers'])

    # Tính toán cho events
    true_positives_events = 0
    for pred_event in pred_events:
        if any(is_event_similar(pred_event, true_event, threshold) for true_event in true_events):
            true_positives_events += 1

    false_positives_events = len(pred_events) - true_positives_events
    false_negatives_events = len(true_events) - true_positives_events

    # Tính toán cho negative triggers
    true_positives_neg = len(pred_neg_triggers.intersection(true_neg_triggers))
    false_positives_neg = len(pred_neg_triggers) - true_positives_neg
    false_negatives_neg = len(true_neg_triggers) - true_positives_neg

    # Tổng hợp kết quả
    true_positives = true_positives_events + true_positives_neg
    false_positives = false_positives_events + false_positives_neg
    false_negatives = false_negatives_events + false_negatives_neg

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [16]:

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

## Kết quả chạy các mẫu (dừng ở mẫu 17)

In [20]:
def run_each_samples(sample_data):
  text = create_text_from_sample(sample_data)
  candidates, ground_truth = extract_events_and_neg_triggers(sample_data)

  prompt = create_prompt(text, candidates)
  #print(prompt)
  def process_func(prompt):
    messages = [
      {"role": "system", "content": prompt },
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    outputs = model.generate(
      input_ids,
      max_new_tokens=3000,
      eos_token_id=terminators,
      do_sample=False,
      temperature=0.0,
    )
    response = outputs[0][input_ids.shape[-1]:]
    llm_output = tokenizer.decode(response, skip_special_tokens=True)
    llm_results = parse_llm_output(llm_output)
    return llm_results
  llm_results = process_chunks(text, candidates, 10, process_func)
  return calculate_event_f1(llm_results, ground_truth)

In [21]:
total_precision = 0
total_recall = 0
total_f1 = 0
total_count = 0
for i, sample in enumerate(sample_set, 1):
    try:
      precision, recall, f1 = run_each_samples(sample)
    except Exception as e:
      print(e)
      continue
    total_precision += precision
    total_recall += recall
    total_f1 += f1
    print(total_precision)
    print(total_recall)
    print(total_f1)
    total_count += 1
total_precision /= total_count
total_recall /= total_count
total_f1 /= total_count

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `

0.7045454545454546
0.5849056603773585
0.6391752577319587


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end gene

1.6093073593073592
1.2886093640810623
1.4308419243986255


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end gene

...

Expecting value: line 6 column 55 (char 120)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end gene

15.139482931608775
12.423828680625109
13.618642589721134


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end gene

KeyboardInterrupt: 

In [22]:
print(f"avg precision {15.139482931608775/17}")
print(f"avg recall {12.423828680625109/17}")
print(f"avg f1 {13.618642589721134/17}")



avg precision 0.8905578195063986
avg recall 0.7308134518014769
avg f1 0.8010966229247726


In [ ]:
from google.colab import runtime
runtime.unassign()